In [38]:
# 11/14(금) 14:30

# LinearRegression from Scratch


# 구현할 것
- 공부시간과 성적간의 관계를 모델링한다.
    - **머신러닝 모델(모형)이란** 수집한 데이터를 기반으로 입력값(Feature)와 출력값(Target)간의 관계를 하나의 공식으로 정의한 함수이다. 그 공식을 찾는 과정을 **모델링**이라고 한다.
    - 이 예제에서는 공부한 시험시간으로 점수를 예측하는 모델을 정의한다.
    - 입력값과 출력값 간의 관계를 정의할 수있는 다양한 함수(공식)이 있다. 여기에서는 딥러닝과 관계가 있는 **Linear Regression** 을 사용해본다.

# 데이터 확인
- 입력데이터: 공부시간
- 출력데이터: 성적

|공부시간|점수|
|-|-|
|1|20|
|2|40|
|3|60|

우리가 수집한 공부시간과 점수 데이터를 바탕으로 둘 간의 관계를 식으로 정의 할 수 있으면 **내가 몇시간 공부하면 점수를 얼마 받을 수 있는지 예측할 수 있게 된다.**   
수집한 데이터를 기반으로 앞으로 예측할 수있는 모형을 만드는 것이 머신러닝 모델링이다.

  

## 학습(훈련) 데이터셋 만들기
- 모델을 학습시키기 위한 데이터셋을 구성한다.
- 입력데이터와 출력데이터을 각각 다른 행렬로 구성한다.
- 하나의 데이터 포인트의 입력/출력 값은 같은 index에 정의한다.

### 선형회귀 (Linear Regression)
- Feature들의 가중합을 이용해 Target을 추정한다.
- Feature에 곱해지는 가중치(weight)들은 각 Feature가 Target 얼마나 영향을 주는지 영향도가 된다.
    - 음수일 경우는 target값을 줄이고 양수일 경우는 target값을 늘린다.
    - 가중치가 0에 가까울 수록 target에 영향을 주지 않는 feature이고 0에서 멀수록 target에 많은 영향을 준다.
- 모델 학습과정에서 가장 적절한 Feature의 가중치를 찾아야 한다.
      

\begin{align}
&\large \hat{y} = W\cdot X + b\\
&\small \hat{y}: \text{모델추정값}\\
&\small W: \text{가중치}\\
&\small X: \text{Feature(입력값)}\\
&\small b: \text{bias(편향)}
\end{align}



## Train dataset 구성
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `공부시간` 1개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성.   
  이 문제에서 예측할 항목은 `시험점수` 한개이다.

In [39]:
import torch
X_train = torch.tensor([[1], [2], [3]], dtype=torch.float32) # 공부시간
y_train = torch.tensor([[20], [40], [60]], dtype=torch.float32) # 시험점수
X_train.size(), y_train.size()

(torch.Size([3, 1]), torch.Size([3, 1]))

## 파라미터 (weight, bias) 정의
- 학습대상/최적화 대상

In [40]:
weight = torch.randn(1, 1, requires_grad=True)  # feature에 곱할 값. (가중치)
                            # (1: 입력 feature의 개수, 1: 출력값(예측값)의 개수)
bias = torch.randn(1, requires_grad=True)
print(weight.size(), bias.size())
print(weight)
print(bias)

torch.Size([1, 1]) torch.Size([1])
tensor([[1.5398]], requires_grad=True)
tensor([0.1440], requires_grad=True)


In [41]:
# 추론
pred = X_train @ weight + bias
pred

tensor([[1.6838],
        [3.2237],
        [4.7635]], grad_fn=<AddBackward0>)

In [42]:
# 정답
y_train

tensor([[20.],
        [40.],
        [60.]])

In [43]:
# 오차 계산
loss = torch.mean((y_train - pred)**2, dim=0)
loss

tensor([1579.6830], grad_fn=<MeanBackward1>)

In [44]:
# gradient 계산 -> loss에 대한 weight, bias의 순간 변화율 (gradient)를 계산.
loss.backward()

In [45]:
weight.grad, bias.grad

(tensor([[-171.7188]]), tensor([-73.5526]))

In [46]:
lr = 0.1
weight.data = weight.data - weight.grad * lr
weight.data

tensor([[18.7117]])

In [47]:
bias.data = bias.data - bias.grad * lr
bias.data

tensor([7.4993])

### 모델링

In [48]:
# 모델 정의
weight = torch.randn(1, 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# 추론 모델
def linear_model(X):
    return X @ weight + bias

In [49]:
# loss (오차) 계산 함수
def mse_loss_fn (pred, y):
    return torch.mean((pred-y)**2)

### 학습
1. 모델을 이용해 추정한다.
   - pred = model(input)
1. loss를 계산한다.
   - loss = loss_fn(pred, target)
1. 계산된 loss를 파라미터에 대해 미분하여 계산한 gradient 값을 각 파라미터에 저장한다.
   - loss.backward()
1. optimizer를 이용해 파라미터를 update한다.
   - optimizer.step()  
1. 파라미터의 gradient(미분값)을 0으로 초기화한다.
   - optimizer.zero_grad()
- 위의 단계를 반복한다.   

In [50]:
epochs = 2000  # train set을 몇 번 학습하는지. 
lr = 0.01
for epoch in range(epochs):
    # 1. 추론 (예측)
    pred = linear_model (X_train)
    # 2. 오차 계산
    loss = mse_loss_fn(pred, y_train)
    # 3. 파라미터들 (weight, bias)에 대한 gradient 계산
    loss.backward()
    # 4. 파라미터 업데이트
    weight.data = weight.data - lr * weight.grad
    bias.data = bias.data - lr * bias.grad
    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None

    # 마지막 epoch, 100 epoch마다 loss를 출력 (성능 개선 상황을 모니터링)
    if epoch % 100 == 0 and epoch == (epoch-1):
        print(f"[{epoch}/{epochs}] - loss: {loss.item():.4f}")

In [51]:
weight.data, bias.data

(tensor([[19.9715]]), tensor([0.0648]))

In [52]:
# 추론
with torch.no_grad():
    new_X = torch.tensor([[5],[7]], dtype=torch.float32)
    pred = linear_model (new_X)
    print(pred)


tensor([[ 99.9223],
        [139.8653]])


# 다중 입력, 다중 출력
- 다중입력: Feature가 여러개인 경우
- 다중출력: Output 결과가 여러개인 경우

다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의.  
[참조](https://www.kaggle.com/code/aakashns/pytorch-basics-linear-regression-from-scratch)


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|

```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.
        - `개별 과일를 예측하기 위한 weight들 @ feature들` 의 계산 결과를  **Node, Unit, Neuron** 이라고 한다.
        - 두 과일에 대한 Unit들을 묶어서 **Layer** 라고 한다.
- 목적은 우리가 수집한 train 데이터셋을 이용해 **정확한 예측을 위한 weight와 bias 들**을 찾는 것이다.

## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `온도, 강수량, 습도` 세개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다. 이 문제에서 예측할 항목은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [53]:
#  input: 생산환경 (temp, rainfall, humidity) : (5, 3)
environs = [
    [73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70]
]

# Targets: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = [
    [56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]
]

In [54]:
import torch
# Dataset을 torch.Tensor로 생성
X = torch.tensor(environs, dtype=torch.float32)
y = torch.tensor(apple_orange_output, dtype=torch.float32)
X.shape, y.shape

(torch.Size([5, 3]), torch.Size([5, 2]))

In [55]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

## weight와 bias
- weight: 각 feature들이 생산량에 영향을 주었는지의 가중치로 feature에 곱해줄 값.
    - 사과, 오렌지의 생산량을 구해야 하므로 가중치가 두개가 된다.
    - weight의 shape: `(3, 2)`
- bias는 모든 feature들이 0일때 생산량이 얼마일지를 나타내는 값으로 feature와 weight간의 가중합 결과에 더해줄 값이다.
    - 사과, 오렌지의 생산량을 구하므로 bias가 두개가 된다.
    - bias의 shape: `(2, )`

### Linear Regression model
모델은 weights `w`와 inputs `x`의 내적(dot product)한 값에 bias `b`를 더하는 함수.

$$
\hspace{2.5cm} X \hspace{1.1cm} \cdot \hspace{1.2cm} W \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\cdot
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$


$w_{11},\,w_{12},\,w_{13}$: 사과 생산량 계산시 각 feature들(생산환경)에 곱할 가중치   <br>
$w_{21},\,w_{22},\,w_{23}$: 오렌지 생산량 계산시 각 feature들(생산환경)에 곱할 가중치    
</center>

<center>
<img src="https://raw.githubusercontent.com/kgmyhGit/image_resource/main/deeplearning/figures/3_unit_layer.png">
</center>

In [56]:
# weight/bias 를 정의 -> 초기값은 random 값을 이용해서 생성.
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

weight.size(), bias.size()
# weight: (3:input feature개수  ,  2:output 개수)
# bias  : (2:output 개수, )

(torch.Size([3, 2]), torch.Size([2]))

In [57]:
weight

tensor([[ 0.2992, -1.5517],
        [-0.9342, -2.4584],
        [ 0.5761,  0.4386]], requires_grad=True)

In [58]:
bias

tensor([ 1.3955, -0.5672], requires_grad=True)

In [59]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [60]:
# 한번 학습(최적화)
## 추론
pred = X @ weight + bias

In [61]:
print(pred.size())
pred

torch.Size([5, 2])


tensor([[ -14.5876, -259.6920],
        [ -16.7233, -330.0375],
        [ -64.3521, -439.5497],
        [  13.0530, -248.3207],
        [ -27.3219, -312.9360]], grad_fn=<AddBackward0>)

In [62]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [63]:
## loss 계산(MSE)
loss = torch.mean((pred - y)**2) # 전체 추론한 결과의 평균오차를 계산. dim = 0이라 생략. 
loss

tensor(95549.4219, grad_fn=<MeanBackward0>)

In [64]:
# loss를 가지고 파라미터들(weight들, bias들)의 gradient 계산.
loss.backward()

In [65]:
weight.data

tensor([[ 0.2992, -1.5517],
        [-0.9342, -2.4584],
        [ 0.5761,  0.4386]])

In [66]:
weight.grad

tensor([[ -7980.4316, -34402.0078],
        [-10158.7656, -38095.3945],
        [ -5875.5112, -23152.6855]])

In [67]:
bias.data, bias.grad

(tensor([ 1.3955, -0.5672]), tensor([ -98.1864, -410.1072]))

In [68]:
# 파라미터 업데이트
lr = 0.00001
weight.data = weight.data - lr * weight.grad
bias.data = bias.data - lr * bias.grad

In [69]:
weight.data

tensor([[ 0.3790, -1.2077],
        [-0.8327, -2.0775],
        [ 0.6349,  0.6702]])

In [ ]:
bias.data

In [70]:
## 업데이트된 파라미터로 추정 -> loss 계산
pred2 = X @ weight + bias
loss2 = torch.mean((pred2 - y)**2)

In [71]:
print(loss.item(), loss2.item())

95549.421875 64843.8359375


##  모델링

In [72]:
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

## 모델 정의
def model(X):
    return X @ weight + bias

## loss 함수(MSE)
def loss_fn(pred, y):
    return torch.mean((pred - y)**2) # 전체 오차의 평균.

In [73]:
X.shape

torch.Size([5, 3])

In [74]:
epochs = 5000
lr = 0.00001  # 1e-5
for epoch in range(epochs):
    # 1. 추론
    pred = model(X)
    
    # 2. loss 계산
    loss = loss_fn(pred, y)
    # 3. 파라미터 들의 gradient 계산
    loss.backward()
    # 4. 파라미터 업데이트
    weight.data = weight.data - lr * weight.grad
    bias.data = bias.data - lr * bias.grad
    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None
    ## 100 epoch, 마지막 epoch에서 loss를 출력 => 학습 과정 log를 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item():.5f}")

[0001/5000] - 7291.89209
[0101/5000] - 779.96088
[0201/5000] - 228.29524
[0301/5000] - 71.95204
[0401/5000] - 26.80233
[0501/5000] - 13.07234
[0601/5000] - 8.33968
[0701/5000] - 6.28163
[0801/5000] - 5.09849
[0901/5000] - 4.26405
[1001/5000] - 3.61306
[1101/5000] - 3.08456
[1201/5000] - 2.64934
[1301/5000] - 2.28919
[1401/5000] - 1.99067
[1501/5000] - 1.74307
[1601/5000] - 1.53767
[1701/5000] - 1.36728
[1801/5000] - 1.22592
[1901/5000] - 1.10864
[2001/5000] - 1.01134
[2101/5000] - 0.93063
[2201/5000] - 0.86365
[2301/5000] - 0.80809
[2401/5000] - 0.76200
[2501/5000] - 0.72376
[2601/5000] - 0.69204
[2701/5000] - 0.66572
[2801/5000] - 0.64388
[2901/5000] - 0.62577
[3001/5000] - 0.61074
[3101/5000] - 0.59827
[3201/5000] - 0.58792
[3301/5000] - 0.57934
[3401/5000] - 0.57222
[3501/5000] - 0.56632
[3601/5000] - 0.56142
[3701/5000] - 0.55735
[3801/5000] - 0.55398
[3901/5000] - 0.55117
[4001/5000] - 0.54885
[4101/5000] - 0.54693
[4201/5000] - 0.54533
[4301/5000] - 0.54400
[4401/5000] - 0.54290


In [75]:
# 새로운 데이터로 추론
p = model(X)
p

tensor([[ 57.2655,  70.4142],
        [ 82.1816, 100.6142],
        [118.5912, 132.9187],
        [ 21.0446,  36.9956],
        [102.0218, 119.1773]], grad_fn=<AddBackward0>)

In [76]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [ ]:
new_x = torch.tensor([68, 82, 56], dtype=torch.float32)
new_x = new_x.unsqueeze(dim=0) # dummy 축 하나 늘려서 데이터 shape 맞춰줌.
new_x.shape

torch.Size([1, 3])

In [92]:
with torch.no_grad():
    p = model(new_x)

print(p)

tensor([[80.8438, 95.6138]])


# pytorch built-in 모델을 사용해 Linear Regression 구현

In [79]:
inputs = torch.tensor(
    [[73, 67, 43], 
     [91, 88, 64], 
     [87, 134, 58], 
     [102, 43, 37], 
     [69, 96, 70]], dtype=torch.float32)

In [80]:
targets = torch.tensor(
    [[56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]], dtype=torch.float32)

## torch.nn.Linear
Pytorch는 torch.nn.Linear 클래스를 통해 Linear Regression 모델을 제공.  
torch.nn.Linear에 입력 feature의 개수와 출력 값의 개수를 지정하면 random 값으로 초기화한 weight와 bias들을 생성해 모델을 구성.
- `torch.nn.Linear(input feature의 개수 , output 값의 개수)`

## Optimizer와 Loss 함수 정의
- **Optimizer**: 계산된 gradient값을 이용해 파라미터들을 업데이트 하는 함수
- **Loss 함수**: 정답과 모델이 예측한 값사이의 차이(오차)를 계산하는 함수.
  - 모델을 최적화하는 것은 이 함수의 값을 최소화하는 것을 말한다. 
- `torch.optim` 모듈에 다양한 Optimizer 클래스가 구현돼있다.
- `torch.nn` 또는 `torch.nn.functional` 모듈에 다양한 Loss 함수가 제공. 

In [81]:
# 선형회귀 모델을 정의. torch.nn.Linear 클래스
import torch
import torch.nn as nn

model = nn.Linear(3, 2)  # 3: input feature 개수, 2: output 수

In [93]:
model.weight

Parameter containing:
tensor([[-0.4015,  0.8481,  0.6864],
        [-0.3049,  0.7944,  0.9083]], requires_grad=True)

In [94]:
model.bias

Parameter containing:
tensor([0.1618, 0.3350], requires_grad=True)

In [96]:
model.parameters()

<generator object Module.parameters at 0x112805540>

In [97]:
for a in model.parameters():
    print(a)

Parameter containing:
tensor([[-0.4015,  0.8481,  0.6864],
        [-0.3049,  0.7944,  0.9083]], requires_grad=True)
Parameter containing:
tensor([0.1618, 0.3350], requires_grad=True)


In [95]:
# loss 함수
loss_fn = torch.nn.MSELoss()  # 클래스
loss_fn = torch.nn.functional.mse_loss # 함수 

In [ ]:
# optimizer (torch.optim 모듈에 정의): weight.data = weight.data - lr * weight.grad
optimizer = torch.optim.SGD(
    model.parameters(), # 최적화 대상 파라미터들을 model에서 조회해서 전달.
    lr = 0.00001,       # Learning Rate
)

In [84]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1200, -0.3668, -0.5452],
         [-0.2150,  0.1306, -0.5634]], requires_grad=True),
 Parameter containing:
 tensor([-0.0009,  0.1378], requires_grad=True)]

## Model Train

In [98]:
epochs = 5000

for epoch in range(epochs):
    # 추론
    pred = model(inputs)  

    # loss 계산
    loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)

    # gradient 계산
    loss.backward()

    # 파라미터 업데이트: optimizer.step()
    optimizer.step()  # w = weight - lr * g

    # 파라미터 초기화 w.grad=None, b.grad=None
    optimizer.zero_grad()
    
    # 현재 epoch 학습 결과를 log로 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

[0001/5000] - 0.5230908393859863
[0101/5000] - 0.5230861306190491
[0201/5000] - 0.5230798721313477
[0301/5000] - 0.5230717062950134
[0401/5000] - 0.5230673551559448
[0501/5000] - 0.5230611562728882
[0601/5000] - 0.5230522155761719
[0701/5000] - 0.5230481028556824
[0801/5000] - 0.5230424404144287
[0901/5000] - 0.5230342149734497
[1001/5000] - 0.5230284929275513
[1101/5000] - 0.5230208039283752
[1201/5000] - 0.523013710975647
[1301/5000] - 0.5230101346969604
[1401/5000] - 0.5230016112327576
[1501/5000] - 0.5229915380477905
[1601/5000] - 0.5229874849319458
[1701/5000] - 0.5229817628860474
[1801/5000] - 0.5229753255844116
[1901/5000] - 0.5229697227478027
[2001/5000] - 0.5229617953300476
[2101/5000] - 0.5229564309120178
[2201/5000] - 0.5229493975639343
[2301/5000] - 0.5229436159133911
[2401/5000] - 0.5229352116584778
[2501/5000] - 0.5229302644729614
[2601/5000] - 0.5229254961013794
[2701/5000] - 0.5229184031486511
[2801/5000] - 0.5229124426841736
[2901/5000] - 0.5229045152664185
[3001/5000]

In [86]:
# 추론 => gradient 계산을 할 필요가 없다. ==> grad_fn을 만들 필요가 없다. 그래서 torch.no_grad() 블록에서 추론 작업을 실행한다.
with torch.no_grad():
    pred = model(inputs)

In [87]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [88]:
pred

tensor([[ 57.1715,  70.3419],
        [ 82.1559, 100.5835],
        [118.7989, 133.1033],
        [ 21.1111,  37.0539],
        [101.8147, 118.9932]])

In [89]:
# 학습 로직을 함수 구현
def train(inputs, targets, epochs, model, loss_fn, optimizer):

    for epoch in range(epochs):
        # 추론
        pred = model(inputs)
        # loss 계산
        loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)
        # gradient 계산
        loss.backward()
        # 파라미터 업데이트: optimizer.step()
        optimizer.step()
        # 파라미터 초기화 w.grad=None, b.grad=None
        optimizer.zero_grad()
        # 현재 epoch 학습 결과를 log로 출력
        if epoch % 100 == 0 or epoch == epochs-1:
            print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

In [90]:
model = nn.Linear(3, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [91]:
train(inputs, targets, 5000, model, nn.functional.mse_loss, optimizer)

[0001/5000] - 6789.21484375
[0101/5000] - 0.9640099406242371
[0201/5000] - 0.5908817052841187
[0301/5000] - 0.5337421894073486
[0401/5000] - 0.5249785780906677
[0501/5000] - 0.5236315131187439
[0601/5000] - 0.5234196186065674
[0701/5000] - 0.5233834385871887
[0801/5000] - 0.5233713388442993
[0901/5000] - 0.5233637094497681
[1001/5000] - 0.5233577489852905
[1101/5000] - 0.5233473777770996
[1201/5000] - 0.5233420133590698
[1301/5000] - 0.5233361721038818
[1401/5000] - 0.5233300924301147
[1501/5000] - 0.5233222842216492
[1601/5000] - 0.5233172178268433
[1701/5000] - 0.5233108401298523
[1801/5000] - 0.5233049392700195
[1901/5000] - 0.5232974886894226
[2001/5000] - 0.5232869982719421
[2101/5000] - 0.5232809782028198
[2201/5000] - 0.5232780575752258
[2301/5000] - 0.5232701897621155
[2401/5000] - 0.5232641696929932
[2501/5000] - 0.52325838804245
[2601/5000] - 0.5232487916946411
[2701/5000] - 0.523244321346283
[2801/5000] - 0.5232349634170532
[2901/5000] - 0.5232311487197876
[3001/5000] - 0.52